# Imports and bronze table reading

In [0]:
import pyspark.sql.functions as func
from pyspark.sql.types import StringType, DateType
from pyspark.sql.functions import col, trim, length

In [0]:
df = spark.table("workspace.bronze.crm_sales_details")
#df.display()

# Transformations

### Trimming

In [0]:
for field in df.schema.fields:
    if isinstance(field.dataType, StringType):
        df = df.withColumn(field.name, trim(col(field.name)))

### Cleaning Dates

In [0]:
df = (
    df
    .withColumn(
        "sls_order_dt",
        func.when((col("sls_order_dt") == 0) | (length(col("sls_order_dt")) != 8),None
        ).otherwise(func.to_date(col("sls_order_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_ship_dt",
        func.when((col("sls_ship_dt") == 0) | (length(col("sls_ship_dt")) != 8),None
        ).otherwise(func.to_date(col("sls_ship_dt").cast("string"), "yyyyMMdd"))
    )
    .withColumn(
        "sls_due_dt",
        func.when((col("sls_due_dt") == 0) | (length(col("sls_due_dt")) != 8),None
        ).otherwise(func.to_date(col("sls_due_dt").cast("string"), "yyyyMMdd"))
    )
)

### Sales and Price Corrections

In [0]:
df = (
    df.withColumn(
        "sls_price",
        func.when(
            (col("sls_price").isNull()) | (col("sls_price") <= 0),
            func.when(col("sls_quantity") != 0,col("sls_sales") / col("sls_quantity")
            ).otherwise(None)
        ).otherwise(col("sls_price"))
    )
)

### Renaming Columns

In [0]:
RENAME_MAP = {
    "sls_ord_num": "order_number",
    "sls_prd_key": "product_number",
    "sls_cust_id": "customer_id",
    "sls_order_dt": "order_date",
    "sls_ship_dt": "ship_date",
    "sls_due_dt": "due_date",
    "sls_sales": "sales_amount",
    "sls_quantity": "quantity",
    "sls_price": "price"
}
for old_name, new_name in RENAME_MAP.items():
    df = df.withColumnRenamed(old_name, new_name)
     

### Sanity checks

In [0]:
df.limit(10).display()

# Writing Silver Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.silver.crm_sales")

### Sanity checks of silver table

In [0]:
%sql
SELECT * FROM workspace.silver.crm_sales LIMIT 10